# Settings

## Imports

In [ ]:
!pip install torch torchvision transformers pillow tqdm numpy scikit-learn matplotlib seaborn datasets wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
import torchvision
from torchvision import transforms
from transformers import DistilBertModel, DistilBertTokenizer
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler
import numpy as np
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from pathlib import Path
import logging
from datasets import load_dataset
import wandb
import time
import json
import pickle
import copy
import os
from typing import Optional, Dict, Any, List, Tuple

## Config

In [ ]:
# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Configuration class
class Config:
    def __init__(self, **kwargs):
        self.defaults = {
            'batch_size': 64,
            'num_samples': 5000,
            'img_size': 224,
            'embed_dim': 768,
            'save_dir': './experiments',
            'img_dim': 512,
            'text_dim': 768,
            'lr': 1e-4,
            'epochs': 15,
            'temperature': 0.07,
            'viz_frequency': 1,
            'wandb_project': 'adapter-alignment-v2',
            'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),

            # Hyperparameters for studies
            'temperature_range': [0.05, 0.07, 0.1],
            'learning_rates': [1e-4, 1e-3, 1e-2],
            'hidden_dims': [512, 1024, 2048],
            'model_sizes': ['small', 'medium', 'large'],
            'data_sizes': [1000, 5000, 10000],
            'batch_sizes': [32, 64, 128],
        }

        self.defaults.update(kwargs)
        for key, value in self.defaults.items():
            setattr(self, key, value)

        Path(self.save_dir).mkdir(parents=True, exist_ok=True)

    def __str__(self):
        return "\n".join(f"{k}: {v}" for k, v in self.defaults.items())

    def to_dict(self):
        return {k: v for k, v in self.defaults.items()
                if not isinstance(v, torch.device)}


In [ ]:
# Set random seeds
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# Initialize config
config = Config()
logger.info("Configuration initialized with:")
logger.info(str(config))

# Dataset & Models

## Dataset Setup

In [ ]:
class Flickr30kDataset(Dataset):
    def __init__(self, transform=None, tokenizer=None, max_samples: Optional[int] = 5000):
        self.dataset = load_dataset("nlphuji/flickr30k", split='test')
        self.transform = transform

        # Basic tokenizer validation
        if tokenizer is None or not isinstance(tokenizer, DistilBertTokenizer):
            raise ValueError("Valid DistilBertTokenizer must be provided")
        self.tokenizer = tokenizer

        if max_samples:
            self.dataset = self.dataset.select(range(min(max_samples, len(self.dataset))))

        self.pairs = []
        for idx in range(len(self.dataset)):
            item = self.dataset[idx]
            image = item['image']
            captions = item['caption'][:5]  # Use first 5 captions
            self.pairs.extend([(image, caption) for caption in captions])

    def __len__(self) -> int:
        return len(self.pairs)

    def __getitem__(self, idx: int):
        image, caption = self.pairs[idx]
        if self.transform:
            image = self.transform(image)

        # Always tokenize when tokenizer is present (since we validate in __init__)
        encoding = self.tokenizer(
            caption,
            padding='max_length',
            truncation=True,
            max_length=64,
            return_tensors='pt',
        )

        return (
            image,
            encoding['input_ids'].squeeze(),
            encoding['attention_mask'].squeeze()
        )

### Dataloaders' Creation Function

In [ ]:
def get_dataloaders(tokenizer, config):
    """Create train and validation dataloaders"""
    # Define transforms
    transform = transforms.Compose([
        transforms.Resize((config.img_size, config.img_size)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])

    # Create and split dataset
    full_dataset = Flickr30kDataset(
        transform=transform,
        tokenizer=tokenizer,
        max_samples=config.num_samples
    )

    train_size = int(0.8 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(
        full_dataset, [train_size, val_size]
    )

    # Create dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=config.batch_size,
        shuffle=True,
        num_workers=2 if torch.cuda.is_available() else 0,
        pin_memory=torch.cuda.is_available()
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=config.batch_size,
        num_workers=2 if torch.cuda.is_available() else 0,
        pin_memory=torch.cuda.is_available()
    )

    return train_loader, val_loader

#### [TEST] Dataloaders' Creation Function

In [ ]:
def test_dataset():
    print("\n=== Testing Dataset Implementation ===")

    # Get models and tokenizer
    print("1. Initializing models and tokenizer...")
    _, _, tokenizer = get_pretrained_models()

    # Create transforms
    print("\n2. Setting up image transforms...")
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Create dataset
    print("\n3. Creating dataset...")
    dataset = Flickr30kDataset(transform=transform, tokenizer=tokenizer, max_samples=10)
    print(f"Dataset size: {len(dataset)}")

    # Test single item
    print("\n4. Testing single item retrieval...")
    image, input_ids, attention_mask = dataset[0]
    print(f"Image shape: {image.shape}")
    print(f"Input ids shape: {input_ids.shape}")
    print(f"Attention mask shape: {attention_mask.shape}")

    # Test dataloader
    print("\n5. Testing batch loading...")
    loader = DataLoader(dataset, batch_size=4, num_workers=0)
    batch = next(iter(loader))
    print(f"Batch sizes:")
    print(f"- Images: {batch[0].shape}")
    print(f"- Input ids: {batch[1].shape}")
    print(f"- Attention masks: {batch[2].shape}")

    # Verify data ranges
    print("\n6. Verifying data ranges...")
    print(f"Image value range: [{batch[0].min():.3f}, {batch[0].max():.3f}]")
    print(f"Input ids range: [{batch[1].min()}, {batch[1].max()}]")
    print(f"Attention mask values: unique = {torch.unique(batch[2]).tolist()}")

    print("\n=== All tests completed successfully! ===")
    return True

## Models' Setup

### Pretrained Models' Inicialization Function

In [ ]:
def get_pretrained_models():
    """Initialize and return pretrained vision and language models"""
    # Load ResNet18 without the final classification layer
    image_encoder = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)
    image_encoder = nn.Sequential(
        *list(image_encoder.children())[:-2],  # Remove fc and avgpool
        nn.AdaptiveAvgPool2d((1, 1)),  # Global average pooling
        nn.Flatten()  # Flatten to (batch_size, channels)
    )

    # Quick test of image encoder
    with torch.no_grad():
        dummy_input = torch.randn(1, 3, 224, 224)
        output = image_encoder(dummy_input)
        assert output.shape == (1, 512), f"Expected shape (1, 512), got {output.shape}"

    # Initialize language models
    text_encoder = DistilBertModel.from_pretrained('distilbert-base-uncased')
    text_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

    # Freeze encoders
    for param in image_encoder.parameters():
        param.requires_grad = False
    for param in text_encoder.parameters():
        param.requires_grad = False

    return image_encoder, text_encoder, text_tokenizer

#### [TEST] Pretrained Models' Inicialization Function

In [ ]:
# Test model initialization
def test_models():
    print("\n=== Testing Model Initialization ===\n")

    image_encoder, text_encoder, tokenizer = get_pretrained_models()

    # Test image encoder
    print("\nTesting image encoder...")
    dummy_image = torch.randn(1, 3, 224, 224)
    with torch.no_grad():
        img_output = image_encoder(dummy_image)
        print(f"Image encoder output shape: {img_output.shape}")

    # Test tokenizer and text encoder
    print("\nTesting text pipeline...")
    test_text = "This is a test sentence."
    tokens = tokenizer.encode_plus(
        test_text,
        add_special_tokens=True,
        max_length=64,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    with torch.no_grad():
        text_output = text_encoder(
            tokens['input_ids'],
            attention_mask=tokens['attention_mask']
        )[0][:, 0]
        print(f"Text encoder output shape: {text_output.shape}")

    print("\n=== All model tests passed! ===")
    return True

## [TEST] Data & Models

In [ ]:
print("Starting dataset tests...")
test_dataset_success = test_dataset()
print(f"\nTest result: {'✓ Passed' if test_dataset_success else '✗ Failed'}")

print("\nStarting model tests...")
test_model_success = test_models()
print(f"\nTest result: {'✓ Passed' if test_model_success else '✗ Failed'}")

## Dataset & Models Initialization

In [ ]:
# Initialize models and dataloaders
image_encoder, text_encoder, tokenizer = get_pretrained_models()
train_loader, val_loader = get_dataloaders(tokenizer, config)

# Adapters

#### Parent Adapter Class

In [ ]:
class BaseAdapter(nn.Module):
    """Parent class for all adapter modules."""

    def __init__(self, input_dim: int, embed_dim: int):
        """
        Initialize the base adapter with input and embedding dimensions.

        Args:
            input_dim (int): The size of the input features.
            embed_dim (int): The size of the output features (embedding dimension).
        """
        super().__init__()
        self.input_dim = input_dim  # Store the input feature size
        self.embed_dim = embed_dim  # Store the embedding size

    def log_architecture(self, adapter_type: str):
        """
        Log the adapter's architecture details using `wandb`.

        Args:
            adapter_type (str): The type of the adapter (e.g., "linear", "mlp").
        """
        # Use wandb to log the adapter configuration
        wandb.config.update({
            f"{adapter_type}_adapter/input_dim": self.input_dim,
            f"{adapter_type}_adapter/embed_dim": self.embed_dim,
            f"{adapter_type}_adapter/num_parameters": sum(p.numel() for p in self.parameters()),  # Total parameter count
        })

#### Adapter Factory

In [ ]:
class AdapterFactory:
    """Factory class for creating image and text adapters."""

    @staticmethod
    def create_pair(adapter_type: str, img_dim: int, text_dim: int, embed_dim: int, **kwargs):
        """
        Create a pair of adapters (one for image, one for text) based on the specified type.

        Args:
            adapter_type (str): Type of adapter to create ('linear', 'mlp', 'bottleneck').
            img_dim (int): Input dimension for the image adapter.
            text_dim (int): Input dimension for the text adapter.
            embed_dim (int): Embedding (output) dimension for both adapters.
            **kwargs: Additional arguments specific to the adapter type.

        Returns:
            Tuple[nn.Module, nn.Module]: A pair of adapters (image adapter, text adapter).
        """
        logger.info(f"Creating {adapter_type} adapters - Image: {img_dim} → {embed_dim}, Text: {text_dim} → {embed_dim}")

        # Define available adapter types
        ADAPTER_TYPES = {
            'linear': LinearAdapter,
            'mlp': MLPAdapter,
            'bottleneck': BottleneckAdapter
        }

        # Check if the adapter type is valid
        if adapter_type not in ADAPTER_TYPES:
            raise ValueError(f"Unknown adapter type: {adapter_type}")

        # Get the adapter class for the specified type
        AdapterClass = ADAPTER_TYPES[adapter_type]

        # Create image and text adapters
        img_adapter = AdapterClass(img_dim, embed_dim, **kwargs)
        text_adapter = AdapterClass(text_dim, embed_dim, **kwargs)

        # Log adapter architectures to wandb
        img_adapter.log_architecture('image')
        text_adapter.log_architecture('text')

        return img_adapter, text_adapter

def log_adapter_gradients(adapter, adapter_type: str):
    """
    Log gradient statistics for an adapter to wandb.

    Args:
        adapter (nn.Module): The adapter whose gradients are to be logged.
        adapter_type (str): Type of the adapter (e.g., 'image' or 'text').
    """
    for name, param in adapter.named_parameters():
        if param.grad is not None:  # Log only if gradients are available
            wandb.log({
                f'gradients/{adapter_type}/{name}/mean': param.grad.mean().item(),
                f'gradients/{adapter_type}/{name}/std': param.grad.std().item(),
                f'gradients/{adapter_type}/{name}/norm': param.grad.norm().item()
            })


def log_feature_statistics(features: torch.Tensor, prefix: str):
    """
    Log statistics of feature tensors to wandb.

    Args:
        features (torch.Tensor): Feature tensor to analyze.
        prefix (str): Prefix for the logged statistics (e.g., 'image', 'text').
    """
    wandb.log({
        f'{prefix}/mean': features.mean().item(),           # Mean value of the features
        f'{prefix}/std': features.std().item(),             # Standard deviation of the features
        f'{prefix}/norm': features.norm(dim=1).mean().item(), # Mean norm of feature vectors
        f'{prefix}/sparsity': (features == 0).float().mean().item()  # Fraction of zero elements
    })

## Linear Adapter

In [ ]:
class LinearAdapter(BaseAdapter):
    """An adapter that performs a simple linear transformation followed by normalization."""

    def __init__(self, input_dim: int, embed_dim: int):
        """
        Initialize the Linear Adapter with input and embedding dimensions.

        Args:
            input_dim (int): The size of the input features.
            embed_dim (int): The size of the output features (embedding dimension).
        """
        # Call the BaseAdapter initializer
        super().__init__(input_dim, embed_dim)

        # Define the linear transformation (input_dim -> embed_dim)
        self.linear = nn.Linear(input_dim, embed_dim)

        # Define layer normalization for the output
        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Pass input through the linear adapter.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, input_dim).

        Returns:
            torch.Tensor: Output tensor of shape (batch_size, embed_dim).
        """
        # Apply the linear transformation and normalize the result
        return self.norm(self.linear(x))

## MLP Adapter

In [ ]:
class MLPAdapter(BaseAdapter):
    """Adapter using a Multi-layer Perceptron (MLP) architecture."""

    def __init__(self, input_dim: int, embed_dim: int, hidden_dim: int = 1024, num_layers: int = 2):
        """
        Initialize the MLP Adapter.

        Args:
            input_dim (int): The size of the input features.
            embed_dim (int): The size of the output features (embedding dimension).
            hidden_dim (int): The size of the hidden layers (default = 1024).
            num_layers (int): The number of layers in the MLP (default = 2).
        """
        # Call the initialization of the BaseAdapter class
        super().__init__(input_dim, embed_dim)

        # Prepare a list to hold all layers of the MLP
        layers = []

        # Create a list of layer dimensions from input to output
        # For example, if input_dim=256, embed_dim=64, hidden_dim=128, num_layers=3:
        # dims = [256, 128, 128, 64]
        dims = [input_dim] + [hidden_dim] * (num_layers - 1) + [embed_dim]

        # Iterate through pairs of consecutive dimensions (d1 -> d2)
        for d1, d2 in zip(dims[:-1], dims[1:]):
            # Add a fully connected layer, activation, normalization, and dropout
            layers.extend([
                nn.Linear(d1, d2),   # Fully connected layer (d1 inputs -> d2 outputs)
                nn.ReLU(),           # Activation function: ReLU (added to introduce non-linearity)
                nn.LayerNorm(d2),    # Layer normalization for stability
                nn.Dropout(0.1)      # Dropout to reduce overfitting (drop probability = 10%)
            ])

        # Combine all layers into a sequential container
        self.mlp = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Pass input through the MLP.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, input_dim).

        Returns:
            torch.Tensor: Output tensor of shape (batch_size, embed_dim).
        """
        return self.mlp(x)

## Bottleneck Adapter

In [ ]:
class BottleneckAdapter(BaseAdapter):
    """Adapter with a bottleneck design to reduce dimensionality."""

    def __init__(self, input_dim: int, embed_dim: int, bottleneck_dim: int = 256):
        """
        Initialize the Bottleneck Adapter.

        Args:
            input_dim (int): The size of the input features.
            embed_dim (int): The size of the output features (embedding dimension).
            bottleneck_dim (int): The size of the reduced (bottleneck) dimension (default = 256).
        """
        # Call the initialization of the BaseAdapter class
        super().__init__(input_dim, embed_dim)

        # Define the "down" layer: reduces input dimension to the bottleneck dimension
        self.down = nn.Linear(input_dim, bottleneck_dim)

        # Define the "up" layer: maps bottleneck dimension back to the embedding dimension
        self.up = nn.Linear(bottleneck_dim, embed_dim)

        # Layer normalization for the output
        self.norm = nn.LayerNorm(embed_dim)

        # GELU activation function for smooth non-linearity
        self.act = nn.GELU()

        # Dropout layer to prevent overfitting (drop probability = 10%)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Pass input through the bottleneck adapter.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, input_dim).

        Returns:
            torch.Tensor: Output tensor of shape (batch_size, embed_dim).
        """
        # Step 1: Reduce dimensionality using the "down" layer, apply activation, and dropout
        x = self.dropout(self.act(self.down(x)))

        # Step 2: Map back to the embedding dimension using the "up" layer
        x = self.up(x)

        # Step 3: Apply normalization to the final output
        return self.norm(x)

# Loss Function

In [ ]:
class ContrastiveLoss(nn.Module):
    """Contrastive loss with temperature scaling for image-text similarity."""

    def __init__(self, temperature: float = 0.07):
        """
        Initialize the contrastive loss function.

        Args:
            temperature (float): Temperature scaling factor (default = 0.07).
        """
        super().__init__()
        self.temperature = temperature

    def forward(self, image_features: torch.Tensor, text_features: torch.Tensor) -> torch.Tensor:
        """
        Compute contrastive loss between image and text features.

        Args:
            image_features (torch.Tensor): Image feature tensor (batch_size, embed_dim).
            text_features (torch.Tensor): Text feature tensor (batch_size, embed_dim).

        Returns:
            torch.Tensor: Scalar loss value (average bidirectional contrastive loss).
        """
        # Normalize features along the embedding dimension
        image_features = F.normalize(image_features, dim=-1)
        text_features = F.normalize(text_features, dim=-1)

        # Compute the similarity matrix using scaled dot product
        logits = torch.matmul(image_features, text_features.T) / self.temperature

        # Generate labels (diagonal matching)
        labels = torch.arange(len(image_features), device=image_features.device)

        # Compute cross-entropy loss for image-to-text and text-to-image mappings
        loss_i2t = F.cross_entropy(logits, labels)
        loss_t2i = F.cross_entropy(logits.T, labels)

        # Log similarity statistics and individual losses to wandb
        with torch.no_grad():
            wandb.log({
                'similarity/mean': logits.mean().item(),
                'similarity/std': logits.std().item(),
                'similarity/max': logits.max().item(),
                'similarity/min': logits.min().item(),
                'loss/image_to_text': loss_i2t.item(),
                'loss/text_to_image': loss_t2i.item()
            })

        # Return the average of the two losses
        return (loss_i2t + loss_t2i) / 2

# Training and Evaluation

## Image-Text Alignment Class

In [ ]:
class AlignmentTrainer:
    """Trainer class for learning image-text alignment."""

    def __init__(self,
                 image_encoder: nn.Module,
                 text_encoder: nn.Module,
                 image_adapter: nn.Module,
                 text_adapter: nn.Module,
                 config: Config,
                 run_name: str = None):
        """
        Initialize the trainer for image-text alignment.

        Args:
            image_encoder (nn.Module): Pretrained image encoder (e.g., Vision Transformer).
            text_encoder (nn.Module): Pretrained text encoder (e.g., BERT).
            image_adapter (nn.Module): Adapter to modify image features.
            text_adapter (nn.Module): Adapter to modify text features.
            config (Config): Configuration object with training parameters.
            run_name (str): Optional name for the wandb run.
        """
        self.config = config
        self.device = config.device  # Device to use (CPU/GPU)
        self.run_name = run_name

        # Move models to the appropriate device
        self.image_encoder = image_encoder.to(self.device)
        self.text_encoder = text_encoder.to(self.device)
        self.image_adapter = image_adapter.to(self.device)
        self.text_adapter = text_adapter.to(self.device)

        # Ensure encoders are in evaluation mode (frozen)
        self.image_encoder.eval()
        self.text_encoder.eval()

        # Optimizer for adapter parameters
        self.optimizer = torch.optim.AdamW([
            {'params': self.image_adapter.parameters()},
            {'params': self.text_adapter.parameters()}
        ], lr=config.lr)

        # Contrastive loss with temperature scaling
        self.loss_fn = ContrastiveLoss(temperature=config.temperature)

        # Mixed precision training scaler
        self.scaler = torch.amp.GradScaler()

        # Verify encoders are frozen and adapters are trainable
        self._verify_model_states()

        # Initialize metrics for best validation performance
        self.best_val_loss = float('inf')
        self.best_alignment = 0.0

        # Initialize wandb logging
        self._init_wandb()

    def _verify_model_states(self):
        """Ensure encoders are frozen and adapters are trainable."""
        encoders_frozen = (
            all(not p.requires_grad for p in self.image_encoder.parameters()) and
            all(not p.requires_grad for p in self.text_encoder.parameters())
        )
        adapters_trainable = (
            any(p.requires_grad for p in self.image_adapter.parameters()) and
            any(p.requires_grad for p in self.text_adapter.parameters())
        )

        if not encoders_frozen or not adapters_trainable:
            raise ValueError("Encoders must be frozen, and adapters must be trainable.")

    def _init_wandb(self):
        """Initialize wandb for tracking experiments."""
        if wandb.run is None:
            wandb.init(
                project=self.config.wandb_project,
                name=self.run_name,
                config=self.config.to_dict()
            )
        wandb.watch(
            (self.image_adapter, self.text_adapter),
            log="all",
            log_freq=100
        )

    def train_epoch(self, train_loader: DataLoader, epoch: int) -> float:
        """Train the adapters for one epoch.

        Args:
            train_loader (DataLoader): DataLoader for training data.
            epoch (int): Current epoch number.

        Returns:
            float: Average training loss for the epoch.
        """
        if wandb.run is None:
            self._init_wandb()

        self.image_adapter.train()
        self.text_adapter.train()
        total_loss = 0

        print()

        progress_bar = tqdm(train_loader, desc=f"Training Epoch {epoch}/{self.config.epochs}")

        for batch_idx, (images, input_ids, attention_mask) in enumerate(progress_bar):
            with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                images, input_ids, attention_mask = (
                    images.to(self.device),
                    input_ids.to(self.device),
                    attention_mask.to(self.device)
                )

                # Encode features (encoders are frozen)
                with torch.no_grad():
                    img_feats = self.image_encoder(images)
                    text_feats = self.text_encoder(input_ids, attention_mask)[0][:, 0]

                # Adapt features using adapters
                img_adapted = self.image_adapter(img_feats)
                text_adapted = self.text_adapter(text_feats)

                # Log feature statistics
                log_feature_statistics(img_adapted, 'image_features')
                log_feature_statistics(text_adapted, 'text_features')

                # Compute contrastive loss
                loss = self.loss_fn(img_adapted, text_adapted)

            # Backpropagation and optimization
            self.optimizer.zero_grad()
            self.scaler.scale(loss).backward()

            # Log gradients
            log_adapter_gradients(self.image_adapter, 'image')
            log_adapter_gradients(self.text_adapter, 'text')

            self.scaler.step(self.optimizer)
            self.scaler.update()

            total_loss += loss.item()
            progress_bar.set_postfix({'loss': f"{loss.item():.4f}"})

            # Log batch metrics
            wandb.log({
                'batch/loss': loss.item(),
                'batch/learning_rate': self.optimizer.param_groups[0]['lr']
            })

        return total_loss / len(train_loader)

    @torch.no_grad()
    def evaluate(self, val_loader: DataLoader, dinov2_model: nn.Module = None) -> dict:
        """Evaluate adapters on validation data."""
        self.image_adapter.eval()
        self.text_adapter.eval()
        total_loss = 0
        adapter_features, dino_features = [], []

        for images, input_ids, attention_mask in tqdm(val_loader, desc="Evaluating"):
            images, input_ids, attention_mask = (
                images.to(self.device),
                input_ids.to(self.device),
                attention_mask.to(self.device)
            )

            img_feats = self.image_encoder(images)
            text_feats = self.text_encoder(input_ids, attention_mask)[0][:, 0]

            img_adapted = self.image_adapter(img_feats)
            text_adapted = self.text_adapter(text_feats)

            loss = self.loss_fn(img_adapted, text_adapted)
            total_loss += loss.item()

            adapter_features.append(img_adapted.cpu())
            if dinov2_model:
                dino_feats = dinov2_model(images)
                dino_features.append(dino_feats.cpu())

        metrics = {'val_loss': total_loss / len(val_loader)}

        if dinov2_model:
            adapter_features = torch.cat(adapter_features)
            dino_features = torch.cat(dino_features)
            metrics['alignment_score'] = compute_alignment(adapter_features, dino_features)
            self._log_feature_space_visualization(adapter_features, dino_features)

        return metrics

    def _log_feature_space_visualization(self, adapter_features: torch.Tensor, dino_features: torch.Tensor):
        """Visualize and log feature spaces using t-SNE."""
        # Project to smaller dimension (e.g., 128) for visualization
        projection_dim = 128

        adapter_proj = torch.nn.Linear(adapter_features.shape[1], projection_dim)
        dino_proj = torch.nn.Linear(dino_features.shape[1], projection_dim)

        adapter_features_proj = adapter_proj(adapter_features)
        dino_features_proj = dino_proj(dino_features)

        combined = torch.cat([adapter_features_proj, dino_features_proj])
        tsne = TSNE(n_components=2, random_state=42)
        embeddings = tsne.fit_transform(combined.numpy())

        n = len(adapter_features)
        plt.figure(figsize=(10, 10))
        plt.scatter(embeddings[:n, 0], embeddings[:n, 1], label='Adapter', alpha=0.5)
        plt.scatter(embeddings[n:, 0], embeddings[n:, 1], label='DinoV2', alpha=0.5)
        plt.legend()
        plt.title('Feature Space Visualization')
        wandb.log({"feature_space": wandb.Image(plt)})
        plt.close()

## Compute Alignment Function (mutual k-NN)

In [ ]:
def compute_alignment(features1: torch.Tensor, features2: torch.Tensor, k: int = 10) -> float:
    """Compute alignment between two feature sets using mutual nearest neighbors."""
    features1, features2 = F.normalize(features1, dim=1).numpy(), F.normalize(features2, dim=1).numpy()
    nn1, nn2 = NearestNeighbors(n_neighbors=k).fit(features1), NearestNeighbors(n_neighbors=k).fit(features2)
    idx1, idx2 = nn1.kneighbors(features1, return_distance=False), nn2.kneighbors(features2, return_distance=False)

    mutual_nn = np.array([len(set(idx1[i]) & set(idx2[i])) / k for i in range(len(idx1))])
    return mutual_nn.mean()

## Training & Evaluation Loop

In [ ]:
def train_and_evaluate(config: Config,
                       train_loader: DataLoader,
                       val_loader: DataLoader,
                       adapter_type: str,
                       dinov2_model: nn.Module = None,
                       run_name: str = None):
    """Main training loop for image-text alignment."""
    image_encoder, text_encoder, _ = get_pretrained_models()
    img_adapter, text_adapter = AdapterFactory.create_pair(
        adapter_type,
        config.img_dim,
        config.text_dim,
        config.embed_dim
    )

    trainer = AlignmentTrainer(image_encoder, text_encoder, img_adapter, text_adapter, config, run_name)

    for epoch in range(1, config.epochs + 1):
        train_loss = trainer.train_epoch(train_loader, epoch)
        metrics = trainer.evaluate(val_loader, dinov2_model)
        val_loss, alignment_score = metrics['val_loss'], metrics.get('alignment_score', 0.0)

        wandb.log({
            f"{adapter_type}/epoch": epoch,
            f"{adapter_type}/train_loss": train_loss,
            f"{adapter_type}/val_loss": val_loss,
            f"{adapter_type}/alignment_score": alignment_score
        })

        if val_loss < trainer.best_val_loss:
            trainer.best_val_loss = val_loss
            wandb.log({f"{adapter_type}/best_val_loss": val_loss})

        if alignment_score > trainer.best_alignment:
            trainer.best_alignment = alignment_score
            wandb.log({f"{adapter_type}/best_alignment": alignment_score})

    return {
        'trainer': trainer,
        'best_val_loss': trainer.best_val_loss,
        'best_alignment': trainer.best_alignment,
        'final_alignment': alignment_score
    }

# Experimental Framework

**Handles the orchestration of multiple experiments with varying configurations, including:**


*   Running single experiments.
*   Performing ablation studies.
*   Conducting scaling studies.





In [ ]:
class ExperimentRunner:
    def __init__(self, base_config: Config, tokenizer: DistilBertTokenizer):
        self.base_config = base_config
        self.tokenizer = tokenizer
        self.experiment_id = time.strftime('%Y%m%d_%H%M%S')

        # Initialize common resources
        logger.info("Creating dataloaders...")
        self.train_loader, self.val_loader = get_dataloaders(tokenizer, base_config)

        logger.info("Loading DINOv2 model...")
        self.dinov2_model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14').to(base_config.device).eval()

        self.save_dir = Path(base_config.save_dir) / self.experiment_id
        self.save_dir.mkdir(parents=True, exist_ok=True)

    def execute_experiments(self,
                            run_base=True,
                            run_ablation=False,
                            run_scaling=False,
                            adapter_types=['linear', 'mlp', 'bottleneck']):
        """Main method to run selected experiments"""
        results = {}

        if run_base:
            logger.info("Running base experiments...")
            results['base'] = self._run_base_experiments(adapter_types)
        if run_ablation:
            logger.info("Running ablation studies...")
            results['ablation'] = self._run_ablation_study(adapter_types)
        if run_scaling:
            logger.info("Running scaling studies...")
            results['scaling'] = self._run_scaling_study(adapter_types)

        create_analysis_plots(results, self.save_dir)
        save_results(self.save_dir, **results)
        return results

    def _run_single_experiment(self,
                            adapter_type: str,
                            config: Config = None,
                            train_loader: DataLoader = None,
                            val_loader: DataLoader = None,
                            experiment_name: str = None) -> dict:
        """Run a single experiment with given configuration"""
        config = config or self.base_config
        train_loader = train_loader or self.train_loader
        val_loader = val_loader or self.val_loader
        experiment_name = experiment_name or f"{adapter_type}_{self.experiment_id}"

        # Initialize wandb
        wandb.init(
            project=config.wandb_project,
            name=experiment_name,
            config=config.to_dict(),
            group=adapter_type,
            reinit=True
        )

        try:
            results = train_and_evaluate(
                config=config,
                train_loader=train_loader,
                val_loader=val_loader,
                adapter_type=adapter_type,
                dinov2_model=self.dinov2_model
            )

            wandb.log({
                f"final/{adapter_type}/best_val_loss": results['best_val_loss'],
                f"final/{adapter_type}/best_alignment": results['best_alignment'],
                f"final/{adapter_type}/final_alignment": results['final_alignment']
            })

            wandb.finish()
            return results

        except Exception as e:
            logger.error(f"Error in experiment {experiment_name}: {str(e)}")
            wandb.finish(exit_code=1)
            raise

    def run_base_experiments(self, adapter_types: List[str]) -> dict:
        """Run base experiments for specified adapter types"""
        results = {}
        for adapter_type in adapter_types:
            results[adapter_type] = self._run_single_experiment(adapter_type)
        return results

    def run_ablation_study(self, adapter_types: List[str]) -> dict:
        """Run ablation studies for specified adapter types"""
        ablation_results = {}

        for adapter_type in adapter_types:
            # Temperature ablation
            for temp in self.base_config.temperature_range:
                config = copy.deepcopy(self.base_config)
                config.temperature = temp
                ablation_results[f'{adapter_type}_temperature_{temp}'] = self._run_single_experiment(
                    adapter_type=adapter_type,
                    config=config,
                    experiment_name=f'ablation_{adapter_type}_temp_{temp}'
                )

            # Learning rate ablation
            for lr in self.base_config.learning_rates:
                config = copy.deepcopy(self.base_config)
                config.lr = lr
                ablation_results[f'{adapter_type}_lr_{lr}'] = self._run_single_experiment(
                    adapter_type=adapter_type,
                    config=config,
                    experiment_name=f'ablation_{adapter_type}_lr_{lr}'
                )

        return ablation_results

    def run_scaling_study(self, adapter_types: List[str]) -> dict:
        """Run scaling studies for specified adapter types"""
        scaling_results = {}

        for adapter_type in adapter_types:
            # Model size scaling
            for size in self.base_config.model_sizes:
                config = copy.deepcopy(self.base_config)
                config.embed_dim = {'small': 256, 'medium': 512, 'large': 1024}[size]
                scaling_results[f'{adapter_type}_size_{size}'] = self._run_single_experiment(
                    adapter_type=adapter_type,
                    config=config,
                    experiment_name=f'scaling_{adapter_type}_size_{size}'
                )

            # Data size scaling
            for data_size in self.base_config.data_sizes:
                config = copy.deepcopy(self.base_config)
                config.num_samples = data_size

                train_loader, val_loader = get_dataloaders(self.tokenizer, config)
                scaling_results[f'{adapter_type}_data_{data_size}'] = self._run_single_experiment(
                    adapter_type=adapter_type,
                    config=config,
                    train_loader=train_loader,
                    val_loader=val_loader,
                    experiment_name=f'scaling_{adapter_type}_data_{data_size}'
                )

        return scaling_results

    def _update_config_for_experiment(self, original_config: Config, updates: Dict[str, Any]) -> Config:
        """Helper method to create new config with updates"""
        config = copy.deepcopy(original_config)
        for key, value in updates.items():
            setattr(config, key, value)
        return config

    def _log_experiment_metadata(self, experiment_name: str, config: Config):
        """Helper method to log experiment metadata"""
        metadata = {
            'experiment_name': experiment_name,
            'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
            'config': config.to_dict()
        }
        wandb.log({'metadata': metadata})

    def _handle_experiment_failure(self, experiment_name: str, error: Exception):
        """Helper method to handle experiment failures"""
        error_info = {
            'error_type': type(error).__name__,
            'error_message': str(error),
            'experiment_name': experiment_name,
            'timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
        }
        wandb.log({'experiment_error': error_info})
        logger.error(f"Experiment {experiment_name} failed: {str(error)}")

    def _save_experiment_artifacts(self, results: dict):
        """Helper method to save experiment artifacts"""
        with open(self.save_dir / 'results.pkl', 'wb') as f:
            pickle.dump(results, f)

        json_results = {k: {
            'best_val_loss': float(v['best_val_loss']),
            'best_alignment': float(v['best_alignment']),
            'final_alignment': float(v['final_alignment'])
        } for k, v in results.items()}

        with open(self.save_dir / 'results.json', 'w') as f:
            json.dump(json_results, f, indent=4)

## Visualizations

In [ ]:
def create_analysis_plots(results: Dict, save_dir: Path):
    """Create and save analysis plots for all experiment types"""
    # Initialize wandb if not already running
    if wandb.run is None:
        wandb.init(project="adapter-alignment-v2", name="analysis_plots")

    def _plot_experiment_comparison(results: Dict, name: str):
        if not results:
            return

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

        # Plot alignment scores
        scores = {k: v['best_alignment'] for k, v in results.items()}
        sns.barplot(x=list(scores.keys()), y=list(scores.values()), ax=ax1)
        ax1.set_title(f'Best Alignment Scores - {name}')
        ax1.set_ylabel('Alignment Score')
        plt.xticks(rotation=45)

        # Plot validation losses
        losses = {k: v['best_val_loss'] for k, v in results.items()}
        sns.barplot(x=list(losses.keys()), y=list(losses.values()), ax=ax2)
        ax2.set_title(f'Best Validation Losses - {name}')
        ax2.set_ylabel('Loss')
        plt.xticks(rotation=45)

        plt.tight_layout()
        plt.savefig(save_dir / f'{name}_comparison.png')
        wandb.log({f"{name}_comparison": wandb.Image(fig)})
        plt.close()

    def _plot_scaling_trends(scaling_results: Dict):
        if not scaling_results:
            return

        sizes = []
        alignments = []
        losses = []

        for size, result in scaling_results.items():
            sizes.append(size)
            alignments.append(result['best_alignment'])
            losses.append(result['best_val_loss'])

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

        sns.lineplot(x=sizes, y=alignments, marker='o', ax=ax1)
        ax1.set_title('Alignment Score vs Scale')

        sns.lineplot(x=sizes, y=losses, marker='o', ax=ax2)
        ax2.set_title('Validation Loss vs Scale')

        plt.tight_layout()
        plt.savefig(save_dir / 'scaling_trends.png')
        wandb.log({"scaling_trends": wandb.Image(fig)})
        plt.close()

    def _plot_ablation_trends(ablation_results: Dict):
        if not ablation_results:
            return

        fig, axes = plt.subplots(2, 1, figsize=(10, 10))

        # Temperature analysis
        temps = []
        temp_alignments = []
        for config, result in ablation_results.items():
            if 'temperature' in config:
                temps.append(float(config.split('_')[1]))
                temp_alignments.append(result['best_alignment'])

        if temps:
            sns.lineplot(x=temps, y=temp_alignments, marker='o', ax=axes[0])
            axes[0].set_title('Alignment Score vs Temperature')

        # Learning rate analysis
        lrs = []
        lr_alignments = []
        for config, result in ablation_results.items():
            if 'lr' in config:
                lrs.append(float(config.split('_')[1]))
                lr_alignments.append(result['best_alignment'])

        if lrs:
            sns.lineplot(x=lrs, y=lr_alignments, marker='o', ax=axes[1])
            axes[1].set_title('Alignment Score vs Learning Rate')
            axes[1].set_xscale('log')

        plt.tight_layout()
        plt.savefig(save_dir / 'ablation_trends.png')
        wandb.log({"ablation_trends": wandb.Image(fig)})
        plt.close()

    # Plot comparisons for each experiment group
    for group_name, group_results in results.items():
        _plot_experiment_comparison(group_results, group_name)

        if group_name == 'scaling':
            _plot_scaling_trends(group_results)
        elif group_name == 'ablation':
            _plot_ablation_trends(group_results)

    # Close wandb run if we initialized it
    wandb.finish()

# Main

#### Aux function

In [ ]:
def save_results(save_dir: Path, results: Dict):
    """Save experimental results to disk"""
    # Create a copy without unpickleable objects
    pickle_results = {}
    for k, v in results.items():
        pickle_results[k] = {}
        for inner_k, inner_v in v.items():
            if isinstance(inner_v, dict):
                pickle_results[k][inner_k] = {
                    'best_val_loss': inner_v['best_val_loss'],
                    'best_alignment': inner_v['best_alignment'],
                    'final_alignment': inner_v['final_alignment']
                }

    # Save the filtered results
    with open(save_dir / 'results.pkl', 'wb') as f:
        pickle.dump(pickle_results, f)

    # Save human-readable results
    json_results = {k: {
        inner_k: {
            'best_val_loss': float(inner_v['best_val_loss']),
            'best_alignment': float(inner_v['best_alignment']),
            'final_alignment': float(inner_v['final_alignment'])
        }
        for inner_k, inner_v in v.items()
    } for k, v in results.items()}

    with open(save_dir / 'results.json', 'w') as f:
        json.dump(json_results, f, indent=4)

### Main function

In [ ]:
def main(experiments_config=None):
    if experiments_config is None:
        experiments_config = {
            'base_experiments': ['linear', 'mlp', 'bottleneck'],
            'run_ablation': True,
            'ablation_adapters': ['linear', 'mlp', 'bottleneck'],
            'run_scaling': True,
            'scaling_adapters': ['linear', 'mlp', 'bottleneck']
        }

    try:
        wandb.init(project="adapter-alignment-v2", name="main_experiment")

        config = Config()
        image_encoder, text_encoder, tokenizer = get_pretrained_models()
        experiment_runner = ExperimentRunner(config, tokenizer)

        results = {}

        if experiments_config['base_experiments']:
            logger.info("Running base experiments...")
            results['base'] = experiment_runner.run_base_experiments(
                experiments_config['base_experiments']
            )

        if experiments_config['run_ablation']:
            logger.info("Running ablation studies...")
            results['ablation'] = experiment_runner.run_ablation_study(
                experiments_config['ablation_adapters']
            )

        if experiments_config['run_scaling']:
            logger.info("Running scaling experiments...")
            results['scaling'] = experiment_runner.run_scaling_study(
                experiments_config['scaling_adapters']
            )

        wandb.finish()
        return results, experiment_runner.save_dir

    except Exception as e:
        if wandb.run is not None:
            wandb.finish(exit_code=1)
        logger.error(f"Error: {str(e)}")
        raise

# Run

In [ ]:
logging.basicConfig(level=logging.INFO)

results, save_dir = main({
    'base_experiments': ['linear', 'mlp', 'bottleneck'],
    'run_ablation': True,
    'ablation_adapters': ['linear', 'mlp', 'bottleneck'],
    'run_scaling': True,
    'scaling_adapters': ['linear', 'mlp', 'bottleneck']
})

# bottleneck -> large -> 19 GB System Ram

In [ ]:
create_analysis_plots(results, save_dir)

In [ ]:
save_results(save_dir, results)